### 📘 Lesson 4: Building a PyPSA Model

<div style="display: flex; align-items: center; justify-content: space-between;">
  <div>
    <h3>Course presenters</h3>
    <ul>
      <li><strong>Priyesh Gosai</strong> - Energy Systems Modeler and Training Coordinator</li>
      <li><strong>Dr. Ekaterina Fedotova</strong> - Senior Energy Systems Modeler</li>
    </ul>
  </div>
  <div>
    <a href="https://openenergytransition.org/index.html">
      <img src="https://openenergytransition.org/assets/img/oet-logo-red-n-subtitle.png" height="60" alt="OET">
    </a>
  </div>
</div>


##### 🎯 Learning Objectives  



* Introduce participants to the PyPSA toolbox.  
* Provide details of relevant components.  
* Build and solve a simple PyPSA model.  
* Review the data structures for static and time-series data.  
* Analyze the results.  

The content also includes references to other toolboxes such as `numpy`, `pandas`, `matplotlib`, and `plotly`, but only covers functions relevant to a PyPSA workflow.  

📌 Participants unfamiliar with these toolboxes are encouraged to explore online videos or courses for deeper learning. 🎥📚  

---

**Setup Environment**

In [ ]:
# @title "Mount Google Drive and Set Working Directory"

from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set FOLDER and change to the working directory in one step
FOLDER = 'ich-modeling-2025'
os.chdir(f'/content/drive/MyDrive/{FOLDER}')

# Confirm the current working directory
print("Current working directory:", os.getcwd())

In [ ]:
# @title Install latest version of PyPSA with Excel support

import os
from IPython.display import clear_output

# Install required packages
os.system("pip install -q pypsa")
os.system("pip install -q 'pypsa[excel]'")
os.system("pip install -q folium mapclassify")

# Clear the output after installation
clear_output()

# Notify the user
print("✅ All required packages have been installed.\n"
      "💡 To comment this cell, select all the text and press `Ctrl + /`.")

### 📄 **Case Study**

In this notebook, we use the ACDC example included with the PyPSA library.

A detailed walkthrough of this example is available in the official PyPSA documentation at [pypsa.readthedocs.io](pypsa.readthedocs.io), where it is used to demonstrate core modeling concepts and components.

In [ ]:
import pypsa

In [ ]:
network = pypsa.examples.ac_dc_meshed()

In [ ]:
network.plot.explore()

A `snapshot` represents a specific point in time for which the network is simulated.  
- Snapshots can be single timestamps (e.g., `2025-01-01 00:00`) or time series covering hours, days, weeks, or years.  
- They allow modeling of dynamic power system behavior over different time periods.  


In [ ]:
network.snapshots

**Carriers**

☀️ 🌬️ 🔥 💧 🔋 🛢️ 🏭 ⚡ 

A carrier is a label that describes the type of energy or technology associated with a component, such as "solar", "wind", "gas", or "battery". It is used for grouping, visualization, and applying shared attributes like efficiency, emissions, or costs across multiple components.

In [ ]:
network.carriers

🔘 **Busses**

A bus represents a node in the network where energy flows are balanced — it connects components like generators, loads, storage units, and transmission lines. Each bus belongs to a specific carrier (e.g., "AC", "DC", "heat") and enforces Kirchhoff's Current Law to ensure supply equals demand at every time step.

In [ ]:
network.buses

🔗 **Links**

* Links enable controllable, directed power flow between two buses (`bus0 → bus1`).  
* They can have efficiency losses and marginal costs, restricting default flow to one direction.  
* For bidirectional, lossless operation, set `efficiency = 1`, `marginal_cost = 0`, and `p_min_pu = -1`.  
* Links model HVDC interconnections, converters, heat pumps, electrolysers, and other controllable power flows.  






In [ ]:
network.links

🔌 Lines

Lines represent passive, bidirectional power flow between two buses, governed by physical impedance and thermal limits.

They model AC transmission and distribution infrastructure using parameters like x (reactance), s_nom (thermal rating), and length.

Power flows are determined by network topology and Kirchhoff’s Laws, not by control decisions.

Use s_nom_extendable = True for capacity expansion studies and apply type for standard line parameters from components.csv.


In [ ]:
network.lines

**⚡ Generators**

Generators attach to a single bus, converting energy from their `carrier` to the bus `carrier`.  

* Their power output is constrained by `p_nom * p_max_pu` and `p_nom * p_min_pu`.  

* Static limits define dispatchable generators, while time-varying limits model renewables.  

* Time series `p_max_pu` and `p_min_pu` determine availability per snapshot.  

* For unit commitment constraints, refer to the PyPSA documentation. 

Some key variables relevant to this model are given below. 


| Attribute              | Type            | Unit           | Default | Description | Constraint |
|------------------------|----------------|---------------|---------|-------------|------------------|
| `name`              | string         | n/a           | n/a     | Unique name |  |
| `bus`               | string         | n/a           | n/a     | Name of bus to which generator is attached |  |
| `p_nom`            | float          | MW            | 0       | Nominal power for limits in optimization. |  |
| `p_nom_extendable` | boolean        |           | False   | Switch to allow capacity p_nom to be extended in optimization. | |
| `p_min_pu`        | static/series  | per unit      | n/a     | Minimum output per unit of p_nom. | $p_t \geq p_{nom}\times p_{min,pu,t}$ |
| `p_max_pu`        | static/series  | per unit      | 1       | Maximum output per unit of p_nom. | $p_t \leq p_{nom}\times p_{max,pu,t}$ |
| `p_set`           | static/series  | MW            | n/a     | Active power set point (for PF). | $p_t = p_{set}$  |
| `e_sum_min`       | float          | MWh           | -inf    | Minimum total energy produced during optimization horizon. | $\sum p_t \cdot \delta t \leq e_{\max}$
| `e_sum_max`       | float          | MWh           | inf     | Maximum total energy produced during optimization horizon. | $\sum p_t \cdot \delta t \leq e_{\max}$
| `marginal_cost`   | static/series  | currency/MWh  | n/a     | Marginal cost of production of 1 MWh. | |



network.generators

🌞 **Applying `p_max_pu` Constraint on VRE Generators**  

* Variable Renewable Energy (VRE) generators, such as solar and wind, have time-dependent availability limits.  
* The `p_max_pu` constraint, imported as a time-series dataset, determines the maximum power output at each snapshot on a per unit basis.  


Check the number of snapshots using the `len` function.
```
len(network.snapshots)
```

In [ ]:
network.generators_t.p_max_pu.plot()

 🔌 **Loads**

* A load connects to a single bus and consumes power.  
* It can represent electricity demand or other types of loads like hydrogen or heat.  
* If active power is consumed, the load draws from the bus.  
* If reactive power is consumed, the load behaves like an inductor.  
* Loads are essential for demand modeling in power system simulations. ⚡🏠  

In [ ]:
network.loads

In [ ]:
network.loads_t.p_set.plot()

**🔋Batteries**

There are two components for energy storage in PyPSA: Storage Units and Stores.  

* ⚡ Storage Unit  
   * Attaches to a single bus and is used for inter-temporal power shifting with a time-varying state of charge.  
   * The energy capacity is defined as `max_hours * nominal power (MW)`, and it includes charging/discharging efficiencies.  

* 🏭 Store  
   * Connects to a single bus and acts as a fundamental energy storage component without energy conversion.  
   * Controls and optimizes energy capacity size, but power output must be controlled using Link components.  

🔄 Key Differences  
| Feature           | Storage Unit | Store |
|------------------|-------------|-------|
| Power Control | Directly defined | Requires Links |
| Energy Capacity | Fixed as `max_hours * MW` | Optimized independently |
| Marginal Cost | Applies only to discharging | Applies to both charging & discharging |
| Energy Carrier Conversion| Possible | Not possible (inherits from bus) |

Stores are more flexible but require Links for power control, while Storage Units offer a simpler implementation for direct energy storage modeling. ⚙️🔄  

⚡ **Storage Unit**


| Attribute              | Type            | Unit           | Default | Description |
|------------------------|----------------|---------------|---------|-------------|
| `name`              | string         | n/a           | n/a     | Unique name | 
| `bus`               | string         | n/a           | n/a     | Name of bus to which generator is attached |
| `p_nom`            | float          | MW            | 0       | Nominal power for limits in optimization. | 
| `marginal_cost`   | static/series  | currency/MWh  | n/a     | Marginal cost of production of 1 MWh. | 
| `max_hours` | float | h | 1 | Maximum state of charge capacity in terms of hours at full output capacity `p_nom` |  
|`state_of_charge_initial`| float | MWh | 0  | State of charge before the snapshots in the OPF. | 
|`efficiency_store`  | static/series | per unit | 1 | Efficiency of storage on the way into the storage. | 
|`efficiency_dispatch`| static/series | per unit | 1 | Efficiency of storage on the way out of the storage. | 
| `standing_loss`| static/series | per unit | 0 | Losses per hour to state of charge. | 
| `inflow`| static/series | MW | 0 | Inflow to the state of charge, e.g. due to river inflow in hydro reservoir. |


In [ ]:
network.storage_units

Since there are no storage units lets use the `network.add()` function to add a battery. 

In [ ]:
network.add('StorageUnit', 
            name = "London Battery",
            bus = "London",
            p_nom = 1000, 
            state_of_charge_initial = 300, 
            max_hours = 5,
            carrier = "battery")

We might want to change a value. 

Use the `at` method to change a singular value in the DataFrame

In [ ]:
network.generators.at['Manchester Wind', 'p_nom'] = 1000


Use the `loc` method to make complex changes. 

In [ ]:

network.links.loc[network.links.bus1 == 'Norwich DC', 'p_nom_extendable'] = False


Check if we added the battery correctly.

In [ ]:
network.consistency_check()

**Solve Model**

In [ ]:
network.optimize(solver_name='highs')

### Results

In [ ]:
network.generators_t.p.plot()

In [ ]:
network.storage_units_t.p.plot()

In [ ]:
network.storage_units_t.state_of_charge.plot()

### Statistics Library

##### 🔧 Main Methods (You Can Call Like Functions)

| **Function**           | **Description**                                                                 |
|------------------------|----------------------------------------------------------------------------------|
| `energy_balance()`     | Net energy flows (supply - withdrawal) across components                        |
| `supply()`             | Supply only (positive power flows)                                              |
| `withdrawal()`         | Withdrawal only (negative power flows)                                          |
| `curtailment()`        | Curtailment of components with time-varying availability (e.g. VREs)            |
| `capacity_factor()`    | Average output relative to installed capacity                                   |
| `optimal_capacity()`   | Optimal (solved) installed capacity in MW/MWh                                   |
| `installed_capacity()` | Static installed capacity in MW/MWh                                             |
| `expanded_capacity()`  | Difference between optimal and existing capacity                                |
| `capex()`              | Capital expenditure for all components                                          |
| `installed_capex()`    | CAPEX of already built capacity                                                 |
| `expanded_capex()`     | CAPEX of expanded (newly built) capacity                                        |
| `opex()`               | Operational costs (based on marginal cost × dispatch)                           |
| `revenue()`            | Revenue earned from market prices × output                                      |
| `market_value()`       | Revenue per MWh (currency/MWh)                                                  |
| `transmission()`       | Transmission flows between buses/carriers                                       |



##### 🛠 Common Parameters for `network.statistics` Methods

| **Parameter**               | **Type**                     | **Purpose**                                                                 |
|----------------------------|------------------------------|------------------------------------------------------------------------------|
| `comps`                    | `str` or `list`              | Components to include: e.g. `'Generator'`, `'StorageUnit'`                  |
| `aggregate_time`           | `str` / `bool` (`"sum"`, `False`) | Sum/mean over time or return time series                               |
| `aggregate_groups`         | `str` or `callable`          | How to group component groups (`"sum"`, `"mean"`, etc.)                     |
| `aggregate_across_components` | `bool`                  | Combine results across components (e.g., `Generator` + `Load`)              |
| `groupby`                  | `str`, `list`, or `callable` | How to group: `"carrier"`, `"bus_carrier"`, etc.                            |
| `at_port`                  | `bool`, `str`, or `list`     | Include specific ports: e.g., `True`, `"bus1"`                              |
| `carrier`                  | `str` or `list`              | Filter by component carrier (e.g., `"solar"`, `"gas"`)                      |
| `bus_carrier`              | `str` or `list`              | Filter by carrier of the connected bus                                      |
| `nice_names`               | `bool`                       | Use nice names from `n.carriers.nice_name`                                  |
| `drop_zero`                | `bool`                       | Drop zero-valued results                                                    |
| `round`                    | `int`                        | Round output to given number of decimal places                              |
| `direction`                | `"supply"`, `"withdrawal"`   | Used in `energy_balance`/`revenue` to isolate flow direction                |



In [ ]:
network.statistics.energy_balance.plot()

In [ ]:
df = network.statistics.transmission(aggregate_time=False)

In [ ]:
df = network.statistics.transmission(aggregate_time=False)

df.T.plot()

In [ ]:
df = network.statistics.supply()
df

### Export Network

In [ ]:
import os
import pathlib

results_folder = 'results'
case_name = 'lesson6'

os.makedirs(results_folder, exist_ok=True)




csv_result = pathlib.Path(pathlib.Path.cwd(), results_folder, f"{case_name}")
excel_result = pathlib.Path(pathlib.Path.cwd(), results_folder, f"{case_name}.xlsx")
netcdf_result = pathlib.Path(pathlib.Path.cwd(), results_folder, f"{case_name}.nc")
h5_result = pathlib.Path(pathlib.Path.cwd(), results_folder, f"{case_name}.h5")


# Export the network to a folder of CSV files (one file per component like buses, generators, lines, etc.)
network.export_to_csv_folder(csv_result)

# Export the entire network to a single Excel (.xlsx) file with each component in a separate sheet
network.export_to_excel(excel_result)

# Export the network to a NetCDF (.nc) file — useful for efficient storage and cross-platform compatibility
network.export_to_netcdf(netcdf_result)

# Export the network to an HDF5 (.h5) file — fast binary format well-suited for large networks
network.export_to_hdf5(h5_result)



### 
---